# INICIALIZACIÓN

In [ ]:
# @title INSTALACIÓN DE LIBRERÍAS
!pip install rapidfuzz
!pip install googletrans==4.0.0-rc1

In [ ]:
# @title IMPORTACIÓN DE LIBRERÍAS SISTEMA

import pandas as pd
import sys
import importlib

from google.colab import auth
auth.authenticate_user()
print('[google.colab authenticate_user]:\nAuthenticated!\n')

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Agregando la ruta a las funciones personalizadas en el path del sistema
path = '/content/drive/MyDrive/ANIMUM DIRECCION/DIRECCION BI/NOTEBOOKS'
sys.path.append(path)

# Eliminar cualquier instancia previa del módulo en la memoria
if 'dpm_SQL' in sys.modules:
    del sys.modules['dpm_SQL']

# Importar y forzar la recarga del módulo con importlib
import dpm_SQL
importlib.reload(dpm_SQL)

# Importar las funciones desde el módulo actualizado
from dpm_SQL import (
    map_elements_dynamic_df,
    GBQ_tables_fields_info_df,
    fields_name_format,
    SQL_generate_cleaning_str,
    SQL_generate_academic_date_str,
    SQL_generate_join_tables_str,
    SQL_generate_CPL_to_contacts_str,
    SQL_generate_deal_ordinal_str,
    SQL_generate_BI_view_str
)

print("Módulo dpm_SQL actualizado correctamente.")


[google.colab authenticate_user]:
Authenticated!

Mounted at /content/drive
Módulo dpm_SQL actualizado correctamente.


## GBQ DATASETS SCHEMA

In [ ]:
# @title GBQ INFO GLOBAL
%%time

# Configuración
project_id = 'animum-dev-datawarehouse' # @param {"type":"string"}
datasets = [ ]                          # @param {"type":"string"}
include_tables = True                   # @param {"type":"boolean"}

config = {
    'project_id': project_id,
    'datasets': datasets,
    'include_tables': include_tables
}

# Llamada a la nueva función
full_info_from_GBQ_df = GBQ_tables_fields_info_df(config)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Visualizar

display(full_info_from_GBQ_df)


# Restaurar configuración de pandas
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title GBQ DATASETS
print("Datasets disponibles:\n")
for dataset in full_info_from_GBQ_df['dataset_id'].unique():
    print(f"{dataset}")


Datasets disponibles:

BOMojo_staging_01
FAds_staging_v01
GAds_staging_v01
IMDb_raw_01
IMDb_staging_01
facebook_ads_raw_v01
facebook_ads_raw_v01_facebook_ads
facebook_ads_raw_v01_facebook_ads_source
fivetran_metadata
fivetran_metadata_fivetran_platform
fivetran_metadata_stg_fivetran_platform
google_ads_raw_01
google_ads_raw_01_google_ads
google_ads_raw_01_google_ads_source
hubspot_BI_v01
hubspot_raw_v01
hubspot_raw_v01_hubspot
hubspot_raw_v01_stg_hubspot
hubspot_staging_v01
mkt_02st_01
tablas_mapeo
vl_01raw_01


# SELECCIÓN DE TABLAS Y FIELDS

## FACEBOOK ADS

In [ ]:
# @title fads_fields_selected_list

# Dataset y tabla seleccionados
dataset_seleccionado = 'facebook_ads_raw_v01'  # @param {"type":"string"}
tabla_seleccionada = 'demographics_country'        # @param {"type":"string"}

# Filtrar el DataFrame para obtener los fields de la tabla seleccionada
raw_list = full_info_from_GBQ_df[
    (full_info_from_GBQ_df['dataset_id'] == dataset_seleccionado) &
    (full_info_from_GBQ_df['table_name'] == tabla_seleccionada)
]['field_name'].tolist()

# Mostrar la lista de fields
print(f"Fields disponibles en la tabla '{tabla_seleccionada}' del dataset '{dataset_seleccionado}':\n")
for field in raw_list:
    print(f"{field}")



# Listado manual
fads_fields_selected_list = raw_list
fads_fields_selected_list = [
"date",
"country",
"impressions",
"inline_link_clicks",
"cost_per_inline_link_click",
"cpc",
"cpm",
"ctr",
"spend",
"inline_link_click_ctr"
]


Fields disponibles en la tabla 'demographics_country' del dataset 'facebook_ads_raw_v01':

account_id
date
_fivetran_id
country
impressions
inline_link_clicks
reach
cost_per_inline_link_click
cpc
cpm
ctr
frequency
spend
inline_link_click_ctr
_fivetran_synced


In [ ]:
# @title fads_fields_mapped_df

config = {
    "fields_name_raw_list": fads_fields_selected_list,
    "formato_final": "Sentence case",
    "reemplazos": {
        "utm_medium": "UTM_medium",
        "a_o": "año",
        "t_1": "T1",
        "t_2": "T2",
        "t_3": "T3",
        "t_4": "T4"
    },
    "siglas": ["CDM", "IRPF", "ECVT", "CQC", "ID", "IP", "LOL"],

}

fads_fields_mapped_df = fields_name_format(config)

In [ ]:
# @title mapeo_country
mapeo_country = """
CASE
  WHEN country = 'AD' THEN 'Andorra'
  WHEN country = 'AE' THEN 'Emiratos Árabes Unidos'
  WHEN country = 'AF' THEN 'Afganistán'
  WHEN country = 'AG' THEN 'Antigua y Barbuda'
  WHEN country = 'AI' THEN 'Anguila'
  WHEN country = 'AL' THEN 'Albania'
  WHEN country = 'AM' THEN 'Armenia'
  WHEN country = 'AO' THEN 'Angola'
  WHEN country = 'AQ' THEN 'Antártida'
  WHEN country = 'AR' THEN 'Argentina'
  WHEN country = 'AS' THEN 'Samoa Americana'
  WHEN country = 'AT' THEN 'Austria'
  WHEN country = 'AU' THEN 'Australia'
  WHEN country = 'AW' THEN 'Aruba'
  WHEN country = 'AX' THEN 'Islas Åland'
  WHEN country = 'AZ' THEN 'Azerbaiyán'
  WHEN country = 'BA' THEN 'Bosnia y Herzegovina'
  WHEN country = 'BB' THEN 'Barbados'
  WHEN country = 'BD' THEN 'Bangladés'
  WHEN country = 'BE' THEN 'Bélgica'
  WHEN country = 'BF' THEN 'Burkina Faso'
  WHEN country = 'BG' THEN 'Bulgaria'
  WHEN country = 'BH' THEN 'Baréin'
  WHEN country = 'BI' THEN 'Burundi'
  WHEN country = 'BJ' THEN 'Benín'
  WHEN country = 'BL' THEN 'San Bartolomé'
  WHEN country = 'BM' THEN 'Bermudas'
  WHEN country = 'BN' THEN 'Brunéi'
  WHEN country = 'BO' THEN 'Bolivia'
  WHEN country = 'BQ' THEN 'Caribe Neerlandés'
  WHEN country = 'BR' THEN 'Brasil'
  WHEN country = 'BS' THEN 'Bahamas'
  WHEN country = 'BT' THEN 'Bután'
  WHEN country = 'BV' THEN 'Isla Bouvet'
  WHEN country = 'BW' THEN 'Botsuana'
  WHEN country = 'BY' THEN 'Bielorrusia'
  WHEN country = 'BZ' THEN 'Belice'
  WHEN country = 'CA' THEN 'Canadá'
  WHEN country = 'CC' THEN 'Islas Cocos (Keeling)'
  WHEN country = 'CD' THEN 'Rep. Dem. del Congo'
  WHEN country = 'CF' THEN 'Rep. Centroafricana'
  WHEN country = 'CG' THEN 'República del Congo'
  WHEN country = 'CH' THEN 'Suiza'
  WHEN country = 'CI' THEN 'Costa de Marfil'
  WHEN country = 'CK' THEN 'Islas Cook'
  WHEN country = 'CL' THEN 'Chile'
  WHEN country = 'CM' THEN 'Camerún'
  WHEN country = 'CN' THEN 'China'
  WHEN country = 'CO' THEN 'Colombia'
  WHEN country = 'CR' THEN 'Costa Rica'
  WHEN country = 'CU' THEN 'Cuba'
  WHEN country = 'CV' THEN 'Cabo Verde'
  WHEN country = 'CW' THEN 'Curazao'
  WHEN country = 'CX' THEN 'Isla de Navidad'
  WHEN country = 'CY' THEN 'Chipre'
  WHEN country = 'CZ' THEN 'Chequia'
  WHEN country = 'DE' THEN 'Alemania'
  WHEN country = 'DJ' THEN 'Yibuti'
  WHEN country = 'DK' THEN 'Dinamarca'
  WHEN country = 'DM' THEN 'Dominica'
  WHEN country = 'DO' THEN 'República Dominicana'
  WHEN country = 'DZ' THEN 'Argelia'
  WHEN country = 'EC' THEN 'Ecuador'
  WHEN country = 'EE' THEN 'Estonia'
  WHEN country = 'EG' THEN 'Egipto'
  WHEN country = 'EH' THEN 'Sáhara Occidental'
  WHEN country = 'ER' THEN 'Eritrea'
  WHEN country = 'ES' THEN 'España'
  WHEN country = 'ET' THEN 'Etiopía'
  WHEN country = 'FI' THEN 'Finlandia'
  WHEN country = 'FJ' THEN 'Fiyi'
  WHEN country = 'FK' THEN 'Islas Malvinas'
  WHEN country = 'FM' THEN 'Micronesia'
  WHEN country = 'FO' THEN 'Islas Feroe'
  WHEN country = 'FR' THEN 'Francia'
  WHEN country = 'GA' THEN 'Gabón'
  WHEN country = 'GB' THEN 'Reino Unido'
  WHEN country = 'GD' THEN 'Granada'
  WHEN country = 'GE' THEN 'Georgia'
  WHEN country = 'GF' THEN 'Guayana Francesa'
  WHEN country = 'GG' THEN 'Guernsey'
  WHEN country = 'GH' THEN 'Ghana'
  WHEN country = 'GI' THEN 'Gibraltar'
  WHEN country = 'GL' THEN 'Groenlandia'
  WHEN country = 'GM' THEN 'Gambia'
  WHEN country = 'GN' THEN 'Guinea'
  WHEN country = 'GP' THEN 'Guadalupe'
  WHEN country = 'GQ' THEN 'Guinea Ecuatorial'
  WHEN country = 'GR' THEN 'Grecia'
  WHEN country = 'GS' THEN 'Islas Georgias del Sur'
  WHEN country = 'GT' THEN 'Guatemala'
  WHEN country = 'GU' THEN 'Guam'
  WHEN country = 'GW' THEN 'Guinea-Bisáu'
  WHEN country = 'GY' THEN 'Guyana'
  WHEN country = 'HK' THEN 'Hong Kong'
  WHEN country = 'HM' THEN 'Islas Heard y McDonald'
  WHEN country = 'HN' THEN 'Honduras'
  WHEN country = 'HR' THEN 'Croacia'
  WHEN country = 'HT' THEN 'Haití'
  WHEN country = 'HU' THEN 'Hungría'
  WHEN country = 'ID' THEN 'Indonesia'
  WHEN country = 'IE' THEN 'Irlanda'
  WHEN country = 'IL' THEN 'Israel'
  WHEN country = 'IM' THEN 'Isla de Man'
  WHEN country = 'IN' THEN 'India'
  WHEN country = 'IO' THEN 'Terr. Británico Oc. Índico'
  WHEN country = 'IQ' THEN 'Iraq'
  WHEN country = 'IR' THEN 'Irán'
  WHEN country = 'IS' THEN 'Islandia'
  WHEN country = 'IT' THEN 'Italia'
  WHEN country = 'JE' THEN 'Jersey'
  WHEN country = 'JM' THEN 'Jamaica'
  WHEN country = 'JO' THEN 'Jordania'
  WHEN country = 'JP' THEN 'Japón'
  WHEN country = 'KE' THEN 'Kenia'
  WHEN country = 'KG' THEN 'Kirguistán'
  WHEN country = 'KH' THEN 'Camboya'
  WHEN country = 'KI' THEN 'Kiribati'
  WHEN country = 'KM' THEN 'Comoras'
  WHEN country = 'KN' THEN 'San Cristóbal y Nieves'
  WHEN country = 'KP' THEN 'Corea del Norte'
  WHEN country = 'KR' THEN 'Corea del Sur'
  WHEN country = 'KW' THEN 'Kuwait'
  WHEN country = 'KY' THEN 'Islas Caimán'
  WHEN country = 'KZ' THEN 'Kazajistán'
  WHEN country = 'LA' THEN 'Laos'
  WHEN country = 'LB' THEN 'Líbano'
  WHEN country = 'LC' THEN 'Santa Lucía'
  WHEN country = 'LI' THEN 'Liechtenstein'
  WHEN country = 'LK' THEN 'Sri Lanka'
  WHEN country = 'LR' THEN 'Liberia'
  WHEN country = 'LS' THEN 'Lesoto'
  WHEN country = 'LT' THEN 'Lituania'
  WHEN country = 'LU' THEN 'Luxemburgo'
  WHEN country = 'LV' THEN 'Letonia'
  WHEN country = 'LY' THEN 'Libia'
  WHEN country = 'MA' THEN 'Marruecos'
  WHEN country = 'MC' THEN 'Mónaco'
  WHEN country = 'MD' THEN 'Moldavia'
  WHEN country = 'ME' THEN 'Montenegro'
  WHEN country = 'MF' THEN 'San Martín (Francia)'
  WHEN country = 'MG' THEN 'Madagascar'
  WHEN country = 'MH' THEN 'Islas Marshall'
  WHEN country = 'MK' THEN 'Macedonia del Norte'
  WHEN country = 'ML' THEN 'Malí'
  WHEN country = 'MM' THEN 'Birmania (Myanmar)'
  WHEN country = 'MN' THEN 'Mongolia'
  WHEN country = 'MO' THEN 'Macao'
  WHEN country = 'MP' THEN 'Islas Marianas del Norte'
  WHEN country = 'MQ' THEN 'Martinica'
  WHEN country = 'MR' THEN 'Mauritania'
  WHEN country = 'MS' THEN 'Montserrat'
  WHEN country = 'MT' THEN 'Malta'
  WHEN country = 'MU' THEN 'Mauricio'
  WHEN country = 'MV' THEN 'Maldivas'
  WHEN country = 'MW' THEN 'Malaui'
  WHEN country = 'MX' THEN 'México'
  WHEN country = 'MY' THEN 'Malasia'
  WHEN country = 'MZ' THEN 'Mozambique'
  WHEN country = 'NA' THEN 'Namibia'
  WHEN country = 'NC' THEN 'Nueva Caledonia'
  WHEN country = 'NE' THEN 'Níger'
  WHEN country = 'NF' THEN 'Isla Norfolk'
  WHEN country = 'NG' THEN 'Nigeria'
  WHEN country = 'NI' THEN 'Nicaragua'
  WHEN country = 'NL' THEN 'Países Bajos'
  WHEN country = 'NO' THEN 'Noruega'
  WHEN country = 'NP' THEN 'Nepal'
  WHEN country = 'NR' THEN 'Nauru'
  WHEN country = 'NU' THEN 'Niue'
  WHEN country = 'NZ' THEN 'Nueva Zelanda'
  WHEN country = 'OM' THEN 'Omán'
  WHEN country = 'PA' THEN 'Panamá'
  WHEN country = 'PE' THEN 'Perú'
  WHEN country = 'PF' THEN 'Polinesia Francesa'
  WHEN country = 'PG' THEN 'Papúa Nueva Guinea'
  WHEN country = 'PH' THEN 'Filipinas'
  WHEN country = 'PK' THEN 'Pakistán'
  WHEN country = 'PL' THEN 'Polonia'
  WHEN country = 'PM' THEN 'San Pedro y Miquelón'
  WHEN country = 'PN' THEN 'Islas Pitcairn'
  WHEN country = 'PR' THEN 'Puerto Rico'
  WHEN country = 'PS' THEN 'Palestina'
  WHEN country = 'PT' THEN 'Portugal'
  WHEN country = 'PW' THEN 'Palaos'
  WHEN country = 'PY' THEN 'Paraguay'
  WHEN country = 'QA' THEN 'Catar'
  WHEN country = 'RE' THEN 'Reunión'
  WHEN country = 'RO' THEN 'Rumanía'
  WHEN country = 'RS' THEN 'Serbia'
  WHEN country = 'RU' THEN 'Rusia'
  WHEN country = 'RW' THEN 'Ruanda'
  WHEN country = 'SA' THEN 'Arabia Saudí'
  WHEN country = 'SB' THEN 'Islas Salomón'
  WHEN country = 'SC' THEN 'Seychelles'
  WHEN country = 'SD' THEN 'Sudán'
  WHEN country = 'SE' THEN 'Suecia'
  WHEN country = 'SG' THEN 'Singapur'
  WHEN country = 'SH' THEN 'Santa Elena'
  WHEN country = 'SI' THEN 'Eslovenia'
  WHEN country = 'SJ' THEN 'Islas Svalbard y Jan Mayen'
  WHEN country = 'SK' THEN 'Eslovaquia'
  WHEN country = 'SL' THEN 'Sierra Leona'
  WHEN country = 'SM' THEN 'San Marino'
  WHEN country = 'SN' THEN 'Senegal'
  WHEN country = 'SO' THEN 'Somalia'
  WHEN country = 'SR' THEN 'Surinam'
  WHEN country = 'SS' THEN 'Sudán del Sur'
  WHEN country = 'ST' THEN 'Santo Tomé y Príncipe'
  WHEN country = 'SV' THEN 'El Salvador'
  WHEN country = 'SX' THEN 'Sint Maarten'
  WHEN country = 'SY' THEN 'Siria'
  WHEN country = 'SZ' THEN 'Esuatini'
  WHEN country = 'TC' THEN 'Islas Turcas y Caicos'
  WHEN country = 'TD' THEN 'Chad'
  WHEN country = 'TF' THEN 'Territorios Australes Franceses'
  WHEN country = 'TG' THEN 'Togo'
  WHEN country = 'TH' THEN 'Tailandia'
  WHEN country = 'TJ' THEN 'Tayikistán'
  WHEN country = 'TK' THEN 'Tokelau'
  WHEN country = 'TL' THEN 'Timor Oriental'
  WHEN country = 'TM' THEN 'Turkmenistán'
  WHEN country = 'TN' THEN 'Túnez'
  WHEN country = 'TO' THEN 'Tonga'
  WHEN country = 'TR' THEN 'Turquía'
  WHEN country = 'TT' THEN 'Trinidad y Tobago'
  WHEN country = 'TV' THEN 'Tuvalu'
  WHEN country = 'TW' THEN 'Taiwán'
  WHEN country = 'TZ' THEN 'Tanzania'
  WHEN country = 'UA' THEN 'Ucrania'
  WHEN country = 'UG' THEN 'Uganda'
  WHEN country = 'UM' THEN 'Islas menores alejadas de EE.UU.'
  WHEN country = 'US' THEN 'Estados Unidos'
  WHEN country = 'UY' THEN 'Uruguay'
  WHEN country = 'UZ' THEN 'Uzbekistán'
  WHEN country = 'VA' THEN 'Ciudad del Vaticano'
  WHEN country = 'VC' THEN 'San Vicente y las Granadinas'
  WHEN country = 'VE' THEN 'Venezuela'
  WHEN country = 'VG' THEN 'Islas Vírgenes Británicas'
  WHEN country = 'VI' THEN 'Islas Vírgenes de EE.UU.'
  WHEN country = 'VN' THEN 'Vietnam'
  WHEN country = 'VU' THEN 'Vanuatu'
  WHEN country = 'WF' THEN 'Wallis y Futuna'
  WHEN country = 'WS' THEN 'Samoa'
  WHEN country = 'YE' THEN 'Yemen'
  WHEN country = 'YT' THEN 'Mayotte'
  WHEN country = 'ZA' THEN 'Sudáfrica'
  WHEN country = 'ZM' THEN 'Zambia'
  WHEN country = 'ZW' THEN 'Zimbabue'
  ELSE country
END
"""

## GOOGLE ADS

In [ ]:
# @title gads_campaign_report_fields_selected_list

# Dataset y tabla seleccionados
dataset_seleccionado = 'google_ads_raw_01_google_ads'  # @param {"type":"string"}
tabla_seleccionada = 'google_ads__campaign_report'        # @param {"type":"string"}

# Filtrar el DataFrame para obtener los fields de la tabla seleccionada
raw_list = full_info_from_GBQ_df[
    (full_info_from_GBQ_df['dataset_id'] == dataset_seleccionado) &
    (full_info_from_GBQ_df['table_name'] == tabla_seleccionada)
]['field_name'].tolist()

# Mostrar la lista de fields
print(f"Fields disponibles en la tabla '{tabla_seleccionada}' del dataset '{dataset_seleccionado}':\n")
for field in raw_list:
    print(f"{field}")


# Listado manual
gads_campaign_report_fields_selected_list = [
"date_day",
"campaign_name",
"campaign_id",
"advertising_channel_type",
"advertising_channel_subtype",
"status",
"spend",
"clicks",
"impressions",
"conversions",
"conversions_value"
]


Fields disponibles en la tabla 'google_ads__campaign_report' del dataset 'google_ads_raw_01_google_ads':

source_relation
date_day
account_name
account_id
currency_code
campaign_name
campaign_id
advertising_channel_type
advertising_channel_subtype
status
spend
clicks
impressions
conversions
conversions_value
view_through_conversions


In [ ]:
# @title gads_campaign_report_fields_mapped_df

config = {
    "fields_name_raw_list": gads_campaign_report_fields_selected_list,
    "formato_final": "Sentence case",
    "reemplazos": {
        "utm_medium": "UTM_medium",
        "a_o": "año",
        "t_1": "T1",
        "t_2": "T2",
        "t_3": "T3",
        "t_4": "T4"
    },
    "siglas": ["CDM", "IRPF", "ECVT", "CQC", "ID", "IP", "LOL"],

}

gads_campaign_report_fields_mapped_df = fields_name_format(config)

In [ ]:
# @title gads_campaign_criterion_history_fields_selected_list

# Dataset y tabla seleccionados
dataset_seleccionado = 'google_ads_raw_01'  # @param {"type":"string"}
tabla_seleccionada = 'campaign_criterion_history'        # @param {"type":"string"}

# Filtrar el DataFrame para obtener los fields de la tabla seleccionada
raw_list = full_info_from_GBQ_df[
    (full_info_from_GBQ_df['dataset_id'] == dataset_seleccionado) &
    (full_info_from_GBQ_df['table_name'] == tabla_seleccionada)
]['field_name'].tolist()

# Mostrar la lista de fields
print(f"Fields disponibles en la tabla '{tabla_seleccionada}' del dataset '{dataset_seleccionado}':\n")
for field in raw_list:
    print(f"{field}")


# Listado manual
gads_campaign_criterion_history_fields_selected_list = [
"campaign_id",
"bid_modifier",
"geo_target_constant_id",
"negative",
"status",
"updated_at"
]


Fields disponibles en la tabla 'campaign_criterion_history' del dataset 'google_ads_raw_01':

_fivetran_start
campaign_id
id
_fivetran_active
_fivetran_end
_fivetran_synced
age_range_type
bid_modifier
carrier_country_code
carrier_name
content_label_type
device_type
display_name
gender_type
geo_target_constant_id
income_range_type
ip_block_ip_address
keyword_match_type
keyword_text
language_code
language_name
mobile_app_category_constant_id
mobile_app_category_constant_name
mobile_application_app_id
mobile_application_name
mobile_device_id
negative
operating_system_version_id
parental_status_type
placement_url
status
topic_constant_id
type
updated_at
user_interest_id
user_list_id
youtube_channel_id
youtube_video_id


In [ ]:
# @title gads_campaign_criterion_history_fields_mapped_df

config = {
    "fields_name_raw_list": gads_campaign_criterion_history_fields_selected_list,
    "formato_final": "Sentence case",
    "reemplazos": {
        "utm_medium": "UTM_medium",
        "a_o": "año",
        "t_1": "T1",
        "t_2": "T2",
        "t_3": "T3",
        "t_4": "T4"
    },
    "siglas": ["CDM", "IRPF", "ECVT", "CQC", "ID", "IP", "LOL"],

}

gads_campaign_criterion_history_fields_mapped_df = fields_name_format(config)

In [ ]:
# @title gads_geo_target_fields_selected_list

# Dataset y tabla seleccionados
dataset_seleccionado = 'google_ads_raw_01'  # @param {"type":"string"}
tabla_seleccionada = 'geo_target'        # @param {"type":"string"}

# Filtrar el DataFrame para obtener los fields de la tabla seleccionada
raw_list = full_info_from_GBQ_df[
    (full_info_from_GBQ_df['dataset_id'] == dataset_seleccionado) &
    (full_info_from_GBQ_df['table_name'] == tabla_seleccionada)
]['field_name'].tolist()

# Mostrar la lista de fields
print(f"Fields disponibles en la tabla '{tabla_seleccionada}' del dataset '{dataset_seleccionado}':\n")
for field in raw_list:
    print(f"{field}")


# Listado manual
gads_geo_target_fields_selected_list = [
"id",
"canonical_name",
"name",
"target_type"
]


Fields disponibles en la tabla 'geo_target' del dataset 'google_ads_raw_01':

id
_fivetran_synced
canonical_name
country_code
name
parent_geo_target_id
status
target_type


In [ ]:
# @title gads_geo_target_fields_mapped_df

config = {
    "fields_name_raw_list": gads_geo_target_fields_selected_list,
    "formato_final": "Sentence case",
    "reemplazos": {
        "utm_medium": "UTM_medium",
        "a_o": "año",
        "t_1": "T1",
        "t_2": "T2",
        "t_3": "T3",
        "t_4": "T4"
    },
    "siglas": ["CDM", "IRPF", "ECVT", "CQC", "ID", "IP", "LOL"],

}

gads_geo_target_fields_mapped_df = fields_name_format(config)

## HUBSPOT

In [ ]:
# @title hs_contact_fields_selected_list

# Listado manual
hs_contact_fields_selected_list = [
"id",
"_fivetran_deleted",
"_fivetran_synced",
"property_a_o",
"property_address",
"property_autonomo_fecha_de_alta",
"property_autonomo_porcentaje_retencion_irpf",
"property_categorizacion_tutor_otros",
"property_city",
"property_closedate",
"property_comentarios_tutor_coordinador",
"property_comentarios_tutor_recruiter",
"property_correos_de_marketing_tasa_apertura",
"property_correos_de_marketing_tasa_de_clic",
"property_country",
"property_createdate",
"property_date_of_birth",
"property_days_to_close",
"property_dia",
"property_disponibilidad_a_colaborar",
"property_ecvt",
"property_ecvt_control",
"property_ecvt_coordinador",
"property_email",
"property_email_tutor_zoho",
"property_email_tutor_zoho_estado",
"property_es_potencial_tutor",
"property_fecha_de_nacimiento",
"property_fecha_de_nacimiento_operar",
"property_fecha_para_volver_a_valorar_al_candidato",
"property_firstname",
"property_gender",
"property_honorario_de_tutor",
"property_honorarios_cqc",
"property_hs_analytics_average_page_views",
"property_hs_analytics_first_timestamp",
"property_hs_analytics_first_visit_timestamp",
"property_hs_analytics_last_timestamp",
"property_hs_analytics_last_visit_timestamp",
"property_hs_analytics_num_page_views",
"property_hs_analytics_source",
"property_hs_email_click",
"property_hs_email_delivered",
"property_hs_last_sales_activity_timestamp",
"property_hs_last_sales_activity_type",
"property_hs_latest_meeting_activity",
"property_hs_latest_sequence_ended_date",
"property_hs_latest_sequence_enrolled",
"property_hs_latest_sequence_enrolled_date",
"property_hs_latest_sequence_finished_date",
"property_hs_latest_sequence_unenrolled_date",
"property_hs_latest_source",
"property_hs_object_id",
"property_ip_city",
"property_ip_country",
"property_ip_country_code",
"property_ip_de_pais_para_informe",
"property_ip_state",
"property_ip_state_code",
"property_lastname",
"property_num_associated_deals",
"property_phone",
"property_primera_conversion_formulario_fuente",
"property_profesion_actual",
"property_profesion_actual_rango",
"property_profesion_anterior_no_relacionada",
"property_profesion_anterior_no_relacionada_listado",
"property_profesion_anterior_relacionada",
"property_profesion_anterior_relacionada_listado",
"property_tiempo_de_dedicacion",
"property_tiempo_de_dedicacion_listado",
"property_utm_medium",
"property_nombre_fiscal",
"property_provincia",
"property_state",
"property_propuesta_2_tipo_sesion_en_directo",
"property_profesion_anterior_rol_clonado_"
]


In [ ]:
# @title hs_contact_fields_mapped_df

config = {
    "fields_name_raw_list": hs_contact_fields_selected_list,
    "formato_final": "Sentence case",
    "reemplazos": {
        "utm_medium": "UTM_medium",
        "a_o": "año",
        "t_1": "T1",
        "t_2": "T2",
        "t_3": "T3",
        "t_4": "T4"
    },
    "siglas": ["CDM", "IRPF", "ECVT", "CQC", "ID", "IP", "LOL"],

}

hs_contact_fields_mapped_df = fields_name_format(config)

In [ ]:
# @title hs_deal_fields_selected_list

# Listado manual
hs_deal_fields_selected_list =[
"deal_id",
"_fivetran_deleted",
"_fivetran_synced",
"deal_pipeline_id",
"deal_pipeline_stage_id",
"is_deleted",
"owner_id",
"property_amount",
"property_ano_academico",
"property_ano_convocatoria",
"property_area",
"property_closed_lost_reason",
"property_closedate",
"property_convocatoria",
"property_correos_de_marketing_entregados_al_abrir_negocio",
"property_correos_de_marketing_entregados_al_cerrar_negocio",
"property_correos_de_marketing_entregados_al_cierre_negocio",
"property_correos_de_marketing_entregados_al_crear_negocio",
"property_correos_de_marketing_tasa_de_apertura_al_abrir_negocio",
"property_correos_de_marketing_tasa_de_apertura_al_cerrar_negocio",
"property_correos_de_marketing_tasa_de_apertura_al_cierre_negocio",
"property_correos_de_marketing_tasa_de_apertura_al_crear_negocio",
"property_correos_de_marketing_tasa_de_clic_al_abrir_negocio",
"property_correos_de_marketing_tasa_de_clic_al_cerrar_negocio",
"property_correos_de_marketing_tasa_de_clic_al_cierre_negocio",
"property_correos_de_marketing_tasa_de_clic_al_crear_negocio",
"property_createdate",
"property_days_to_close",
"property_dealname",
"property_disciplina_del_programa",
"property_engagements_last_meeting_booked",
"property_hs_days_to_close_raw",
"property_hs_was_imported",
"property_modulo_abandono",
"property_modulo_inicio",
"property_modulos_del_programa",
"property_nombre",
"property_notas_historico",
"property_notes_last_contacted",
"property_notes_last_updated",
"property_numero_de_vistas_de_pagina_al_abrir_negocio",
"property_numero_de_vistas_de_pagina_al_cerrar_negocio",
"property_numero_de_vistas_de_pagina_blog_al_abrir_negocio",
"property_numero_de_vistas_de_pagina_blog_al_cerrar_negocio",
"property_origen_de_alta_manual",
"property_puntuacion_cdm_al_abrir_negocio",
"property_puntuacion_cdm_al_cerrar_negocio",
"property_puntuacion_cdm_al_cierre_negocio",
"property_puntuacion_cdm_al_crear_negocio",
"property_solicitud_estado",
"property_solicitud_valoracion",
"property_tipo_de_beca",
"property_tipo_de_convalidacion",
"property_tipo_de_matricula",
"property_valor_cobrado",
"property_valor_contrato_matricula",
"property_valor_mermado"
]

In [ ]:
# @title hs_deal_fields_mapped_df

config = {
    "fields_name_raw_list": hs_deal_fields_selected_list,
    "formato_final": "Sentence case",
    "reemplazos": {
        "utm_medium": "UTM_medium",
        "a_o": "año",
        "t_1": "T1",
        "t_2": "T2",
        "t_3": "T3",
        "t_4": "T4"
    },
    "siglas": ["CDM", "IRPF", "ECVT", "CQC", "ID", "IP", "LOL"],

}

hs_deal_fields_mapped_df = fields_name_format(config)

In [ ]:
# @title hs_deal_pipeline_fields_selected_list

# Listado manual
hs_deal_pipeline_fields_selected_list = [
"pipeline_id",
"label"
]

In [ ]:
# @title hs_deal_pipeline_fields_mapped_df

config = {
    "fields_name_raw_list": hs_deal_pipeline_fields_selected_list,
    "formato_final": "Sentence case",
    "reemplazos": {
        "utm_medium": "UTM_medium",
        "a_o": "año",
        "t_1": "T1",
        "t_2": "T2",
        "t_3": "T3",
        "t_4": "T4"
    },
    "siglas": ["CDM", "IRPF", "ECVT", "CQC", "ID", "IP", "LOL"],

}

hs_deal_pipeline_fields_mapped_df = fields_name_format(config)

In [ ]:
# @title hs_deal_pipeline_stage_fields_selected_list

# Listado manual
hs_deal_pipeline_stage_fields_selected_list = [
"stage_id",
"pipeline_id",
"label",
"probability",
"closed_won"
]

In [ ]:
# @title hs_deal_pipeline_stage_fields_mapped_df

config = {
    "fields_name_raw_list": hs_deal_pipeline_stage_fields_selected_list,
    "formato_final": "Sentence case",
    "reemplazos": {
        "utm_medium": "UTM_medium",
        "a_o": "año",
        "t_1": "T1",
        "t_2": "T2",
        "t_3": "T3",
        "t_4": "T4"
    },
    "siglas": ["CDM", "IRPF", "ECVT", "CQC", "ID", "IP", "LOL"],

}

hs_deal_pipeline_stage_fields_mapped_df = fields_name_format(config)

In [ ]:
# @title hs_deal_contact_fields_selected_list

# Listado manual
hs_deal_contact_fields_selected_list = [
"deal_id",
"contact_id",
"_fivetran_synced"
]

In [ ]:
# @title hs_deal_contact_fields_mapped_df

config = {
    "fields_name_raw_list": hs_deal_contact_fields_selected_list,
    "formato_final": "Sentence case",
    "reemplazos": {
        "utm_medium": "UTM_medium",
        "a_o": "año",
        "t_1": "T1",
        "t_2": "T2",
        "t_3": "T3",
        "t_4": "T4"
    },
    "siglas": ["CDM", "IRPF", "ECVT", "CQC", "ID", "IP", "LOL"],

}

hs_deal_contact_fields_mapped_df = fields_name_format(config)

## MARKETING VIEW

In [ ]:
# @title marketing_fields_selected_list

marketing_fields_selected_list = [
"contact_id",
"contact__fivetran_deleted",
"contact_property_a_o",
"contact_property_address",
"contact_property_city",
"contact_property_correos_de_marketing_tasa_apertura",
"contact_property_correos_de_marketing_tasa_de_clic",
"contact_property_country",
"contact_property_createdate",
"contact_property_date_of_birth",
"contact_property_days_to_close",
"contact_property_dia",
"contact_property_email",
"contact_property_fecha_de_nacimiento",
"contact_property_fecha_de_nacimiento_operar",
"contact_property_firstname",
"contact_property_gender",
"contact_property_hs_analytics_average_page_views",
"contact_property_hs_analytics_first_timestamp",
"contact_property_hs_analytics_first_visit_timestamp",
"contact_property_hs_analytics_last_timestamp",
"contact_property_hs_analytics_last_visit_timestamp",
"contact_property_hs_analytics_num_page_views",
"contact_property_hs_analytics_source",
"contact_property_hs_email_click",
"contact_property_hs_email_delivered",
"contact_property_hs_last_sales_activity_timestamp",
"contact_property_hs_last_sales_activity_type",
"contact_property_hs_latest_meeting_activity",
"contact_property_hs_latest_sequence_ended_date",
"contact_property_hs_latest_sequence_enrolled",
"contact_property_hs_latest_sequence_enrolled_date",
"contact_property_hs_latest_sequence_finished_date",
"contact_property_hs_latest_sequence_unenrolled_date",
"contact_property_hs_latest_source",
"contact_property_ip_city",
"contact_property_ip_country",
"contact_property_ip_country_code",
"contact_property_ip_de_pais_para_informe",
"contact_property_ip_state",
"contact_property_ip_state_code",
"contact_property_lastname",
"contact_property_num_associated_deals",
"contact_property_primera_conversion_formulario_fuente",
"contact_property_utm_medium",
"contact_property_nombre_fiscal",
"contact_property_provincia",
"contact_property_state",
"contact_property_createdate_fiscal",
"contact_property_createdate_academico",
"contact__fivetran_synced",
"deal_deal_id",
"deal__fivetran_deleted",
"deal_deal_pipeline_id",
"deal_deal_pipeline_stage_id",
"deal_is_deleted",
"deal_owner_id",
"deal_property_amount",
"deal_property_ano_academico",
"deal_property_ano_convocatoria",
"deal_property_area",
"deal_property_closed_lost_reason",
"deal_property_closedate",
"deal_property_convocatoria",
"deal_property_correos_de_marketing_entregados_al_abrir_negocio",
"deal_property_correos_de_marketing_entregados_al_cerrar_negocio",
"deal_property_correos_de_marketing_entregados_al_cierre_negocio",
"deal_property_correos_de_marketing_entregados_al_crear_negocio",
"deal_property_correos_de_marketing_tasa_de_apertura_al_abrir_negocio",
"deal_property_correos_de_marketing_tasa_de_apertura_al_cerrar_negocio",
"deal_property_correos_de_marketing_tasa_de_apertura_al_cierre_negocio",
"deal_property_correos_de_marketing_tasa_de_apertura_al_crear_negocio",
"deal_property_correos_de_marketing_tasa_de_clic_al_abrir_negocio",
"deal_property_correos_de_marketing_tasa_de_clic_al_cerrar_negocio",
"deal_property_correos_de_marketing_tasa_de_clic_al_cierre_negocio",
"deal_property_correos_de_marketing_tasa_de_clic_al_crear_negocio",
"deal_property_createdate",
"deal_property_days_to_close",
"deal_property_dealname",
"deal_property_disciplina_del_programa",
"deal_property_modulo_abandono",
"deal_property_modulo_inicio",
"deal_property_modulos_del_programa",
"deal_property_nombre",
"deal_property_notas_historico",
"deal_property_numero_de_vistas_de_pagina_al_abrir_negocio",
"deal_property_numero_de_vistas_de_pagina_al_cerrar_negocio",
"deal_property_numero_de_vistas_de_pagina_blog_al_abrir_negocio",
"deal_property_numero_de_vistas_de_pagina_blog_al_cerrar_negocio",
"deal_property_puntuacion_cdm_al_abrir_negocio",
"deal_property_puntuacion_cdm_al_cerrar_negocio",
"deal_property_puntuacion_cdm_al_cierre_negocio",
"deal_property_puntuacion_cdm_al_crear_negocio",
"deal_property_tipo_de_beca",
"deal_property_tipo_de_convalidacion",
"deal_property_tipo_de_matricula",
"deal_property_valor_contrato_matricula",
"deal_property_valor_mermado",
"deal_property_createdate_fiscal",
"deal_property_createdate_academico",
"deal_deal_pipeline",
"deal_deal_pipeline_stage",
"deal__fivetran_synced",
"bridge__fivetran_synced",
"contact_Ads_gads_spend_by_day",
"contact_Ads_gads_impressions_by_day",
"contact_Ads_gads_clicks_by_day",
"contact_Ads_fads_spend_by_day",
"contact_Ads_fads_impressions_by_day",
"contact_Ads_fads_conversions_by_day",
"contact_Ads_fads_clicks_by_day",
"deal_ordinal"
]

In [ ]:
# @title marketing_fields_mapped_df

# Paso 1: Formateo inicial de los campos
config_paso_1 = {
    "fields_name_raw_list": marketing_fields_selected_list,
    "formato_final": "Sentence case",
    "reemplazos": {
        "contact_property_": "C ",
        "contact_": "C ",
        "deal_property_": "D ",
        "deal_deal_": "D ",
        "deal_": "D ",
        "bridge_": "B ",
        "a_o": "año",
        "ano": "año",
        "t_1": "T1",
        "t_2": "T2",
        "t_3": "T3",
        "t_4": "T4"
    },
    "siglas": ["UTM", "CDM", "IRPF", "ECVT", "CQC", "ID", "IP", "LOL"]
}

# Aplicar formateo inicial
fields_name_df_paso_1 = fields_name_format(config_paso_1)

# Paso 2: Ajustes adicionales
config_paso_2 = {
    "fields_name_raw_list": fields_name_df_paso_1.iloc[:, 1].tolist(),
    "formato_final": False,  # Sin formato adicional, solo ajustes de reemplazo
    "reemplazos": {
        "ads fads": "Ads FAds",
        "ads gads": "Ads GAds",
        "fivetran": "Fivetran",
        "hs ": ""
    }
}

# Aplicar ajustes adicionales
fields_name_df_paso_2 = fields_name_format(config_paso_2)

# Combinar resultados del paso 1 y paso 2
marketing_fields_mapped_df = pd.DataFrame({
    fields_name_df_paso_1.columns[0]: fields_name_df_paso_1.iloc[:, 0],
    fields_name_df_paso_2.columns[1]: fields_name_df_paso_2.iloc[:, 1]
})

# Visualizar resultados finales
display(marketing_fields_mapped_df)

,Campo Original,Campo Formateado
0,contact_id,C ID
1,contact__fivetran_deleted,C Fivetran deleted
2,contact_property_a_o,C año
3,contact_property_address,C address
4,contact_property_city,C city
...,...,...
107,contact_Ads_fads_spend_by_day,C Ads FAds spend by day
108,contact_Ads_fads_impressions_by_day,C Ads FAds impressions by day
109,contact_Ads_fads_conversions_by_day,C Ads FAds conversions by day
110,contact_Ads_fads_clicks_by_day,C Ads FAds clicks by day


# SQL SCRIPT TABLES GENERATION

## PARÁMETROS GENERATIVOS

In [ ]:
# @title ** GENERALES

# @markdown **::..**
# @markdown ___
# @markdown > # Generales
##########################################################################################
# Comunes
exclude_records_by_creation_date_bool = False                                    # @param {"type":"boolean"}
exclude_records_by_creation_date_field = "date_day"                              # @param {"type":"string"}
exclude_records_by_creation_date_range = {"from": "2021-06-01"}                 # @param {"type":"string"}

exclude_records_by_keywords_bool = False                                         # @param {"type":"boolean"}
exclude_records_by_keywords_fields = [ ]                                        # Lista vacía es todo # @param {"type":"string"}
exclude_records_by_keywords_words = keywords_descartar = [     "test",     "prueba",     "pruebas",     "ejemplo",     "demo",     "temporal",     "dummy",     "asdf",     "asd",     "lorem",     "ipsum",     "aaa",     "bbbb",     "cccc",     "xxxx",     "zzz",     "null",     "nulo",     "vacio",     "sin datos",     "pendiente",     "eliminar",     "descartar" ]   # @param
fields_to_trim = [ ]                                                            # Lista vacía es todo # @param

# from ipywidgets import interact
# def f(x):
#     return x * 2
# interact(f, x=10)



In [ ]:
# @title ** PARTICULARES

# @markdown **::..**
# @markdown ___
# @markdown > # fads_demographics_country_cleaned
##########################################################################################
fads_cleaned_table_source = "animum-dev-datawarehouse.facebook_ads_raw_v01.demographics_country"  # @param {"type":"string"}
fads_cleaned_table_destination = "animum-dev-datawarehouse.mkt_02st_01.fads_demographics_country_cleaned"  # @param {"type":"string"}
fads_cleaned_fields_destination_prefix = "fads_dc_"  # @param {"type":"string"}
fads_cleaned_fields_mapped_use = False # @param {"type":"boolean"}
fads_cleaned_fields_mapped_df = fads_fields_mapped_df   # @param {"type":"string"}


# @markdown **::..**
# @markdown ___
# @markdown > # gads_campaign_report_cleaned
##########################################################################################
gads_campaign_report_cleaned_table_source = "animum-dev-datawarehouse.google_ads_raw_01_google_ads.google_ads__campaign_report"  # @param {"type":"string"}
gads_campaign_report_cleaned_table_destination = "animum-dev-datawarehouse.mkt_02st_01.gads_campaign_report_cleaned"  # @param {"type":"string"}
gads_campaign_report_cleaned_fields_destination_prefix = "gads_r_"  # @param {"type":"string"}
gads_campaign_report_cleaned_fields_mapped_use = False # @param {"type":"boolean"}
gads_campaign_report_cleaned_fields_mapped_df = gads_campaign_report_fields_mapped_df   # @param {"type":"string"}

# @markdown **::..**
# @markdown ___
# @markdown > # gads_campaign_criterion_history_cleaned
##########################################################################################
gads_campaign_criterion_history_cleaned_table_source = "animum-dev-datawarehouse.google_ads_raw_01.campaign_criterion_history"  # @param {"type":"string"}
gads_campaign_criterion_history_cleaned_table_destination = "animum-dev-datawarehouse.mkt_02st_01.gads_campaign_criterion_history_cleaned"  # @param {"type":"string"}
gads_campaign_criterion_history_cleaned_fields_destination_prefix = "gads_ch_"  # @param {"type":"string"}
gads_campaign_criterion_history_cleaned_fields_mapped_use = False # @param {"type":"boolean"}
gads_campaign_criterion_history_cleaned_fields_mapped_df = gads_campaign_criterion_history_fields_mapped_df   # @param {"type":"string"}

# @markdown **::..**
# @markdown ___
# @markdown > # gads_geo_target_cleaned
##########################################################################################
gads_geo_target_cleaned_table_source = "animum-dev-datawarehouse.google_ads_raw_01.geo_target"  # @param {"type":"string"}
gads_geo_target_cleaned_table_destination = "animum-dev-datawarehouse.mkt_02st_01.gads_geo_target_cleaned"  # @param {"type":"string"}
gads_geo_target_cleaned_fields_destination_prefix = "gads_gt_"  # @param {"type":"string"}
gads_geo_target_cleaned_fields_mapped_use = False # @param {"type":"boolean"}
gads_geo_target_cleaned_fields_mapped_df = gads_geo_target_fields_mapped_df   # @param {"type":"string"}


# @markdown **::..**
# @markdown ___
# @markdown > # hs_contact_cleaned
##########################################################################################
hs_contact_cleaned_table_source = "animum-dev-datawarehouse.hubspot_raw_v01.contact"  # @param {"type":"string"}
hs_contact_cleaned_table_destination = "animum-dev-datawarehouse.hs_02st_01.hs_contact_cleaned"  # @param {"type":"string"}
hs_contact_cleaned_fields_destination_prefix = "hs_c_"  # @param {"type":"string"}
hs_contact_cleaned_fields_mapped_use = False # @param {"type":"boolean"}
hs_contact_cleaned_fields_mapped_df = hs_contact_fields_mapped_df   # @param {"type":"string"}


# @markdown **::..**
# @markdown ___
# @markdown > # hs_deal_cleaned
##########################################################################################
hs_deal_cleaned_table_source = "animum-dev-datawarehouse.hubspot_raw_v01.deal"  # @param {"type":"string"}
hs_deal_cleaned_table_destination = "animum-dev-datawarehouse.mkt_02st_01.hs_deal_cleaned"  # @param {"type":"string"}
hs_deal_cleaned_fields_destination_prefix = "hs_d_"  # @param {"type":"string"}
hs_deal_cleaned_fields_mapped_use = False # @param {"type":"boolean"}
hs_deal_cleaned_fields_mapped_df = hs_deal_fields_mapped_df   # @param {"type":"string"}



# @markdown **::..**
# @markdown ___
# @markdown > # hs_deal_pipeline_cleaned
##########################################################################################
hs_deal_pipeline_cleaned_table_source = "animum-dev-datawarehouse.hubspot_raw_v01.deal_pipeline"  # @param {"type":"string"}
hs_deal_pipeline_cleaned_table_destination = "animum-dev-datawarehouse.mkt_02st_01.hs_deal_pipeline_cleaned"  # @param {"type":"string"}
hs_deal_pipeline_cleaned_fields_destination_prefix = "hs_dp_"  # @param {"type":"string"}
hs_deal_pipeline_cleaned_fields_mapped_use = False # @param {"type":"boolean"}
hs_deal_pipeline_cleaned_fields_mapped_df = hs_deal_pipeline_fields_mapped_df   # @param {"type":"string"}


# @markdown **::..**
# @markdown ___
# @markdown > # hs_deal_pipeline_stage_cleaned
##########################################################################################
hs_deal_pipeline_stage_cleaned_table_source = "animum-dev-datawarehouse.hubspot_raw_v01.deal_pipeline_stage"  # @param {"type":"string"}
hs_deal_pipeline_stage_cleaned_table_destination = "animum-dev-datawarehouse.mkt_02st_01.hs_deal_pipeline_stage_cleaned"  # @param {"type":"string"}
hs_deal_pipeline_stage_cleaned_fields_destination_prefix = "hs_dps_"  # @param {"type":"string"}
hs_deal_pipeline_stage_cleaned_fields_mapped_use = False # @param {"type":"boolean"}
hs_deal_pipeline_stage_cleaned_fields_mapped_df = hs_deal_pipeline_stage_fields_mapped_df   # @param {"type":"string"}



# @markdown **::..**
# @markdown ___
# @markdown > # hs_deal_contact_cleaned
##########################################################################################
hs_deal_contact_cleaned_table_source = "animum-dev-datawarehouse.hubspot_raw_v01.deal_contact"  # @param {"type":"string"}
hs_deal_contact_cleaned_table_destination = "animum-dev-datawarehouse.mkt_02st_01.hs_deal_contact_cleaned"  # @param {"type":"string"}
hs_deal_pipeline_stage_cleaned_fields_destination_prefix = "hs_dc_"  # @param {"type":"string"}
hs_deal_contact_cleaned_fields_mapped_use = False # @param {"type":"boolean"}
hs_deal_contact_cleaned_fields_mapped_df = hs_deal_contact_fields_mapped_df   # @param {"type":"string"}



# @markdown **::..**
# @markdown ___
# @markdown > # view_mkt_contact_deal_CPL

##########################################################################################
view_mkt_contact_deal_CPL_table_source = "animum-dev-datawarehouse.hubspot_staging_v01.*******************"  # @param {"type":"string"}
view_mkt_contact_deal_CPL_table_destination = "animum-dev-datawarehouse.mkt_03BI_v01.view_mkt_contact_deal_CPL"  # @param {"type":"string"}
view_mkt_contact_deal_CPL_fields_mapped_df = marketing_fields_mapped_df  # @param {"type":"string"}
view_mkt_contact_deal_CPL_use_mapped_names = True  # @param {"type":"boolean"}
view_mkt_contact_deal_CPL_match_condition = "T.id = S.id"  # @param {"type":"string"}  # (No se usa en la función, pero se deja si lo necesitas)
view_mkt_contact_deal_CPL_use_date_range = True  # @param {"type":"boolean"}
view_mkt_contact_deal_CPL_creation_date_field = "contact_property_createdate"  # @param {"type":"string"}

range_from = "2021-06-01"  # @param {"type":"string"}
range_to = "2100-01-01"    # @param {"type":"string"}
view_mkt_contact_deal_CPL_date_range = {"from": range_from, "to": range_to}

view_mkt_contact_deal_CPL_exclude_deleted_records_bool = True  # @param {"type":"boolean"}
view_mkt_contact_deal_CPL_exclude_deleted_records_field_name = "_fivetran_deleted"   # @param {"type":"string"}
view_mkt_contact_deal_CPL_exclude_deleted_records_field_value = True   # @param {"type":"boolean"}




## FACEBOOK ADS

In [ ]:
# @title fads_cleaned

fads_cleaned_config = {
    "table_source": fads_cleaned_table_source,
    "table_destination": fads_cleaned_table_destination,
    "fields_mapped_use": fads_cleaned_fields_mapped_use,
    "fields_mapped_df": fads_cleaned_fields_mapped_df,
    "fields_destination_prefix": fads_cleaned_fields_destination_prefix,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}


print(SQL_generate_cleaning_str(fads_cleaned_config))

CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.fads_demographics_country_cleaned` AS
SELECT
  `date` AS `fads_dc_date`,
  `country` AS `fads_dc_country`,
  `impressions` AS `fads_dc_impressions`,
  `inline_link_clicks` AS `fads_dc_inline_link_clicks`,
  `cost_per_inline_link_click` AS `fads_dc_cost_per_inline_link_click`,
  `cpc` AS `fads_dc_cpc`,
  `cpm` AS `fads_dc_cpm`,
  `ctr` AS `fads_dc_ctr`,
  `spend` AS `fads_dc_spend`,
  `inline_link_click_ctr` AS `fads_dc_inline_link_click_ctr`
FROM `animum-dev-datawarehouse.facebook_ads_raw_v01.demographics_country`
WHERE TRUE
;


## GOOGLE ADS

In [ ]:
# @title gads_campaign_report_cleaned

gads_campaign_report_cleaned_config = {
    "table_source": gads_campaign_report_cleaned_table_source,
    "table_destination": gads_campaign_report_cleaned_table_destination,
    "fields_mapped_use": gads_campaign_report_cleaned_fields_mapped_use,
    "fields_mapped_df": gads_campaign_report_cleaned_fields_mapped_df,
    "fields_destination_prefix": gads_campaign_report_cleaned_fields_destination_prefix,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(gads_campaign_report_cleaned_config))


CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.gads_campaign_report_cleaned` AS
SELECT
  `date_day` AS `gads_r_date_day`,
  `campaign_name` AS `gads_r_campaign_name`,
  `campaign_id` AS `gads_r_campaign_id`,
  `advertising_channel_type` AS `gads_r_advertising_channel_type`,
  `advertising_channel_subtype` AS `gads_r_advertising_channel_subtype`,
  `status` AS `gads_r_status`,
  `spend` AS `gads_r_spend`,
  `clicks` AS `gads_r_clicks`,
  `impressions` AS `gads_r_impressions`,
  `conversions` AS `gads_r_conversions`,
  `conversions_value` AS `gads_r_conversions_value`
FROM `animum-dev-datawarehouse.google_ads_raw_01_google_ads.google_ads__campaign_report`
WHERE TRUE
;


In [ ]:
# @title gads_campaign_criterion_history_cleaned

gads_campaign_criterion_history_cleaned_config = {
    "table_source": gads_campaign_criterion_history_cleaned_table_source,
    "table_destination": gads_campaign_criterion_history_cleaned_table_destination,
    "fields_mapped_use": gads_campaign_criterion_history_cleaned_fields_mapped_use,
    "fields_mapped_df": gads_campaign_criterion_history_cleaned_fields_mapped_df,
    "fields_destination_prefix": gads_campaign_criterion_history_cleaned_fields_destination_prefix,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(gads_campaign_criterion_history_cleaned_config))


CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.gads_campaign_criterion_history_cleaned` AS
SELECT
  `campaign_id` AS `gads_ch_campaign_id`,
  `bid_modifier` AS `gads_ch_bid_modifier`,
  `geo_target_constant_id` AS `gads_ch_geo_target_constant_id`,
  `negative` AS `gads_ch_negative`,
  `status` AS `gads_ch_status`,
  `updated_at` AS `gads_ch_updated_at`
FROM `animum-dev-datawarehouse.google_ads_raw_01.campaign_criterion_history`
WHERE TRUE
;


In [ ]:
# @title gads_geo_target_cleaned

gads_geo_target_cleaned_config = {
    "table_source": gads_geo_target_cleaned_table_source,
    "table_destination": gads_geo_target_cleaned_table_destination,
    "fields_mapped_use": gads_geo_target_cleaned_fields_mapped_use,
    "fields_mapped_df": gads_geo_target_cleaned_fields_mapped_df,
    "fields_destination_prefix": gads_geo_target_cleaned_fields_destination_prefix,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(gads_geo_target_cleaned_config))


CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.gads_geo_target_cleaned` AS
SELECT
  `id` AS `gads_gt_id`,
  `canonical_name` AS `gads_gt_canonical_name`,
  `name` AS `gads_gt_name`,
  `target_type` AS `gads_gt_target_type`
FROM `animum-dev-datawarehouse.google_ads_raw_01.geo_target`
WHERE TRUE
;


## HUBSPOT

In [ ]:
# @title hs_contact_cleaned

hs_contact_cleaned_config = {
    "table_source":       hs_contact_cleaned_table_source,
    "table_destination":  hs_contact_cleaned_table_destination,
    "fields_mapped_use":  hs_contact_cleaned_fields_mapped_use,
    "fields_mapped_df":   hs_contact_cleaned_fields_mapped_df,
    "fields_destination_prefix": hs_contact_cleaned_fields_destination_prefix,

    "exclude_records_by_creation_date_bool": exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field": exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords_bool": exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields": exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words": exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

# Generar y mostrar la consulta SQL
print(SQL_generate_cleaning_str(hs_contact_cleaned_config))


CREATE OR REPLACE TABLE `animum-dev-datawarehouse.hs_02st_01.hs_contact_cleaned` AS
SELECT
  `id` AS `hs_c_id`,
  `_fivetran_deleted` AS `hs_c__fivetran_deleted`,
  `_fivetran_synced` AS `hs_c__fivetran_synced`,
  `property_a_o` AS `hs_c_property_a_o`,
  `property_address` AS `hs_c_property_address`,
  `property_autonomo_fecha_de_alta` AS `hs_c_property_autonomo_fecha_de_alta`,
  `property_autonomo_porcentaje_retencion_irpf` AS `hs_c_property_autonomo_porcentaje_retencion_irpf`,
  `property_categorizacion_tutor_otros` AS `hs_c_property_categorizacion_tutor_otros`,
  `property_city` AS `hs_c_property_city`,
  `property_closedate` AS `hs_c_property_closedate`,
  `property_comentarios_tutor_coordinador` AS `hs_c_property_comentarios_tutor_coordinador`,
  `property_comentarios_tutor_recruiter` AS `hs_c_property_comentarios_tutor_recruiter`,
  `property_correos_de_marketing_tasa_apertura` AS `hs_c_property_correos_de_marketing_tasa_apertura`,
  `property_correos_de_marketing_tasa_de_clic` A

In [ ]:
# @title hs_deal_cleaned

hs_deal_cleaned_config = {
    "table_source": hs_deal_cleaned_table_source,
    "table_destination": hs_deal_cleaned_table_destination,
    "fields_mapped_use": hs_deal_cleaned_fields_mapped_use,
    "fields_mapped_df": hs_deal_cleaned_fields_mapped_df,
    "fields_destination_prefix": hs_deal_cleaned_fields_destination_prefix,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(hs_deal_cleaned_config))

CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.hs_deal_cleaned` AS
SELECT
  `deal_id` AS `hs_d_deal_id`,
  `_fivetran_deleted` AS `hs_d__fivetran_deleted`,
  `_fivetran_synced` AS `hs_d__fivetran_synced`,
  `deal_pipeline_id` AS `hs_d_deal_pipeline_id`,
  `deal_pipeline_stage_id` AS `hs_d_deal_pipeline_stage_id`,
  `is_deleted` AS `hs_d_is_deleted`,
  `owner_id` AS `hs_d_owner_id`,
  `property_amount` AS `hs_d_property_amount`,
  `property_ano_academico` AS `hs_d_property_ano_academico`,
  `property_ano_convocatoria` AS `hs_d_property_ano_convocatoria`,
  `property_area` AS `hs_d_property_area`,
  `property_closed_lost_reason` AS `hs_d_property_closed_lost_reason`,
  `property_closedate` AS `hs_d_property_closedate`,
  `property_convocatoria` AS `hs_d_property_convocatoria`,
  `property_correos_de_marketing_entregados_al_abrir_negocio` AS `hs_d_property_correos_de_marketing_entregados_al_abrir_negocio`,
  `property_correos_de_marketing_entregados_al_cerrar_negocio` AS `h

In [ ]:
# @title hs_deal_pipeline_cleaned

hs_deal_cleaned_config = {
    "table_source": hs_deal_pipeline_cleaned_table_source,
    "table_destination": hs_deal_pipeline_cleaned_table_destination,
    "fields_mapped_use": hs_deal_pipeline_cleaned_fields_mapped_use,
    "fields_mapped_df": hs_deal_pipeline_cleaned_fields_mapped_df,
    "fields_destination_prefix": hs_deal_pipeline_cleaned_fields_destination_prefix,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(hs_deal_cleaned_config))

CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.hs_deal_pipeline_cleaned` AS
SELECT
  `pipeline_id` AS `hs_dp_pipeline_id`,
  `label` AS `hs_dp_label`
FROM `animum-dev-datawarehouse.hubspot_raw_v01.deal_pipeline`
WHERE TRUE
;


In [ ]:
# @title hs_deal_pipeline_stage_cleaned

hs_deal_pipeline_stage_cleaned_config = {
    "table_source": hs_deal_pipeline_stage_cleaned_table_source,
    "table_destination": hs_deal_pipeline_stage_cleaned_table_destination,
    "fields_mapped_use": hs_deal_pipeline_stage_cleaned_fields_mapped_use,
    "fields_mapped_df": hs_deal_pipeline_stage_cleaned_fields_mapped_df,
    "fields_destination_prefix": hs_deal_pipeline_stage_cleaned_fields_destination_prefix,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(hs_deal_pipeline_stage_cleaned_config))

CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.hs_deal_pipeline_stage_cleaned` AS
SELECT
  `stage_id` AS `hs_dc_stage_id`,
  `pipeline_id` AS `hs_dc_pipeline_id`,
  `label` AS `hs_dc_label`,
  `probability` AS `hs_dc_probability`,
  `closed_won` AS `hs_dc_closed_won`
FROM `animum-dev-datawarehouse.hubspot_raw_v01.deal_pipeline_stage`
WHERE TRUE
;


In [ ]:
# @title hs_deal_contact_cleaned

hs_deal_contact_cleaned_config = {
    "table_source":       hs_deal_contact_cleaned_table_source,
    "table_destination":  hs_deal_contact_cleaned_table_destination,
    "fields_mapped_use":   hs_deal_contact_cleaned_fields_mapped_use,
    "fields_mapped_df":    hs_deal_contact_fields_mapped_df,


    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(hs_deal_contact_cleaned_config))


CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.hs_deal_contact_cleaned` AS
SELECT
  `deal_id` AS `deal_id`,
  `contact_id` AS `contact_id`,
  `_fivetran_synced` AS `_fivetran_synced`
FROM `animum-dev-datawarehouse.hubspot_raw_v01.deal_contact`
WHERE TRUE
;


## MARKETING VIEW

In [ ]:
# @title view_mkt_contact_deal_CPL
view_mkt_contact_deal_CPL_config = {
    "table_source": view_mkt_contact_deal_CPL_table_source,
    "table_destination": view_mkt_contact_deal_CPL_table_destination,
    "fields_mapped_df": view_mkt_contact_deal_CPL_fields_mapped_df,
    "use_mapped_names": view_mkt_contact_deal_CPL_use_mapped_names,
    # 'match_condition' no lo usa la función, pero se deja por compatibilidad o si lo usas en otra parte
    "match_condition": view_mkt_contact_deal_CPL_match_condition,
    "use_date_range": view_mkt_contact_deal_CPL_use_date_range,
    "creation_date_field": view_mkt_contact_deal_CPL_creation_date_field,
    "date_range": view_mkt_contact_deal_CPL_date_range,
    "exclude_deleted_records_bool": view_mkt_contact_deal_CPL_exclude_deleted_records_bool,
    "exclude_deleted_records_field_name": view_mkt_contact_deal_CPL_exclude_deleted_records_field_name,
    "exclude_deleted_records_field_value": view_mkt_contact_deal_CPL_exclude_deleted_records_field_value,
}

print(SQL_generate_BI_view_str(view_mkt_contact_deal_CPL_config))

CREATE OR REPLACE VIEW `animum-dev-datawarehouse.mkt_03BI_v01.view_mkt_contact_deal_CPL` AS
SELECT
  `contact_id` AS `C ID`,
  `contact__fivetran_deleted` AS `C Fivetran deleted`,
  `contact_property_a_o` AS `C año`,
  `contact_property_address` AS `C address`,
  `contact_property_city` AS `C city`,
  `contact_property_correos_de_marketing_tasa_apertura` AS `C correos de marketing tasa apertura`,
  `contact_property_correos_de_marketing_tasa_de_clic` AS `C correos de marketing tasa de clic`,
  `contact_property_country` AS `C country`,
  `contact_property_createdate` AS `C createdate`,
  `contact_property_date_of_birth` AS `C date of birth`,
  `contact_property_days_to_close` AS `C days to close`,
  `contact_property_dia` AS `C dia`,
  `contact_property_email` AS `C email`,
  `contact_property_fecha_de_nacimiento` AS `C fecha de nacimiento`,
  `contact_property_fecha_de_nacimiento_operar` AS `C fecha de nacimiento operar`,
  `contact_property_firstname` AS `C firstname`,
  `contact_pro